In [236]:
# Import necessary libraries
#from splinter import Browser
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode
from colorama import Fore, init
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import random

In [237]:
init() #Colorama init

random_wait = random.randint(2, 4)

#Red flags según artículo
flags_auto = ("conversable", "al dia", "detalles", "atrasada", "atrasado", "atrasados", "trizado", "trizada", "choques", "choque", "chocado", "deshuase", "desarme", "le falta", "roto", "rota", "dañado", "dañada", "malo", "mala", "repuesto", "repuestos", "no transferible", "multa", "multas", "parte", "partes")

flags = flags_auto

close_locations = ["Valparaiso", "Viña del Mar"]

In [238]:
login = False #True da más detalles de los autos pero puede ser problemático, False da menos detalles pero es más robusto

# Set up search parameters
radius = 250 # TIENE que ser [1, 2, 5, 10, 20, 40, 60, 80, 100, 250, 500]
search_location = "Valparaiso"
default_location = 111068758922766 #Curacaví, esto permite englobar stgo y valpo
min_price = 500000
max_price = 10000000
days_since_listed = 30
availability = f"in%20stock" 
# availability = f"out%20of%20stock" #para revisar lo que ya se vendió
min_km = 1000 #Menos de 1000 km suelen ser autos que omiten ceros
max_km = 300000
min_year = 1990
max_year = 2024
transmission = "manual"
make = "kia "
search_model = ""
exact = "true" #false busca sugerencias, true busca exactamente lo que se pide

In [239]:
# Configurar el servicio de Chrome
service = Service(ChromeDriverManager().install())

# Configurar las opciones de Chrome
options = Options()
# options.add_argument("--headless")  # Para ejecutar Chrome en modo sin cabeza (sin ventana)
options.add_experimental_option("detach", True) # Para que el driver no se cierre al terminar el script
# deshabilitar notificaciones
options.add_argument("--disable-notifications")

# Inicializar el driver de Chrome
driver = webdriver.Chrome(service=service, options=options)
# Set up base url
base_url = "https://www.facebook.com/marketplace/"

#Set up full url
url = f"{base_url}{default_location}/search?availability={availability}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_since_listed}&maxMileage={max_km}&maxYear={max_year}&minMileage={min_km}&minYear={min_year}&transmissionType={transmission}&query={make}{search_model}&exact={exact}"

In [240]:
# Inicio de sesión o no

# Cuenta fake
mail = "roadtonostromo1@gmail.com"
password = "suufreejvrrr"

if login:

    driver.get(base_url)

    mail_input = driver.find_element(By.XPATH ,'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[3]/div/label/div/div/input')

    ActionChains(driver).click(mail_input).perform()

    mail_input.send_keys(mail)

    password_input = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[4]/div/label/div/div/input')

    ActionChains(driver).click(password_input).perform()

    password_input.send_keys(password)

    mail_input.send_keys(Keys.RETURN)

    # Visitar el sitio web
    driver.get(url)

    # Esperar un tiempo breve para que la página se cargue completamente
    time.sleep(random_wait) # 1 segundo

else:
    # Visitar el sitio web
    driver.get(url)

    # Esperar un tiempo breve para que la página se cargue completamente
    time.sleep(random_wait) # 1 segundo

    # Buscar el elemento de la X para ignorar el inicio de sesión
    x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")

    # Hacer clic en la X para ignorar el inicio de sesión
    x_button.click()

Obtener información de la zona de búsqueda

In [241]:
change_location_selector = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]')

# Obtener ubicación actual
current_location = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]/div[1]/div/span').text
print(f"La búsqueda se está realizando en: {current_location}")

# Hacer clic en el selector de ubicación
# change_location_selector.click()

# Cambiar la ubicación de búsqueda
# location = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[2]/div/div/div/div/div/label/div/div[2]/input')

# ActionChains(driver).click(location).perform()

# esperar 1 segundo
time.sleep(random_wait)

La búsqueda se está realizando en: Curacaví, Region Metropolitana, Chile
  · En un radio de 65 kilómetros


In [242]:
#location.clear()

In [243]:
#location.send_keys(search_location)

#location.send_keys(Keys.RETURN)

# Hay que hacer que seleccione una de las opciones del menu desplegable

Facebook cambia los id de algunos elementos, cambiar el radio de busqueda como ultima alternativa ya que se debe volver a buscar el XPATH cada cierto tiempo

In [244]:
""" # buscar selector de radio
# radius_selector = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label')
radius_selector = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label/div/div[1]/div/div')                                               
# Hacer clic en el selector de radio
radius_selector.click()

# Botones de seleccion de radio

elementos = driver.find_elements(By.XPATH, "//div[@class='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xjyslct x9f619 x1ypdohk x78zum5 x1q0g3np x2lah0s xnqzcj9 x1gh759c xdj266r xat24cr x1344otq x1de53dj xz9dl7a xsag5q8 x1n2onr6 x16tdsg8 x1ja2u2z']")

# Selecciona el radio en el menú

for elemento in elementos:
    numero = re.match(r"\d+", elemento.text).group()
    
    if int(numero) == radius:
        elemento.click()
        break

# Buscar el botón de aplicar
apply_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div')

# Hacer clic en el botón de aplicar
apply_button.click() """

' # buscar selector de radio\n# radius_selector = driver.find_element(By.XPATH, \'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label\')\nradius_selector = driver.find_element(By.XPATH, \'/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label/div/div[1]/div/div\')                                               \n# Hacer clic en el selector de radio\nradius_selector.click()\n\n# Botones de seleccion de radio\n\nelementos = driver.find_elements(By.XPATH, "//div[@class=\'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xjyslct x9f619 x1ypdohk x78zum5 x1q0g3np x2lah0s xnqzcj9 x1gh759c xdj266r xat24cr x1344otq x1de53dj xz9dl7a xsag5q8 x1n2onr6 x16tdsg8 x1ja2u2z\']")\n\n# Selecciona el radio en el menú\n\nfor elemento in elementos:\n    num

In [245]:
# SE DEBE PONER EL CODIGO DENTRO DE UN WHILE QUE EVALUE SI SE RECOLECTÒ UNA CANTIDAD
# MINIMA DE AUTOS, SI NO, SE DEBE RECARGAR LA PAGINA HASTA QUE SE CUMPLA

# Número máximo de intentos para detectar si se ha alcanzado el final de la página
max_attempts = 200
# Tiempo máximo para esperar que ocurran cambios en la posición vertical de la barra de desplazamiento
max_scroll_wait = 5  # segundos

# Función para verificar si se ha alcanzado el final de la página
def is_page_end_reached(driver):
    current_scroll_position = driver.execute_script("return window.scrollY;")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(max_scroll_wait)  # Esperar un momento para que se actualice la posición de desplazamiento
    new_scroll_position = driver.execute_script("return window.scrollY;")
    return new_scroll_position == current_scroll_position

# Desplazarse hacia abajo hasta que se haya alcanzado el final de la página o se haya intentado max_attempts veces
attempt = 1
while attempt <= max_attempts:
    if is_page_end_reached(driver):
        break
    attempt += 1
    time.sleep(max_scroll_wait)  # Retraso antes de intentar nuevamente

# Si después de los intentos la página todavía no se ha cargado completamente, espera un tiempo adicional
if attempt > max_attempts:
    WebDriverWait(driver, max_scroll_wait).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

# Ahora la página debe estar completamente cargada


In [246]:
# Obtener el HTML de la página
html = driver.page_source

# Crear un objeto BeautifulSoup a partir del HTML
market_soup = soup(html, 'html.parser')

In [247]:
nombre_html = f"{make}_{transmission}.html"

# Escribir el HTML en un archivo
with open(nombre_html, "w", encoding="utf-8") as file:
    file.write(html)

print(f"El HTML de la página se ha guardado en {nombre_html}")

El HTML de la página se ha guardado en kia _manual.html


In [248]:
""" # Leer html desde archivo

with open(nombre_html, "r", encoding="utf-8") as file:
    html = file.read()

# Crear un objeto BeautifulSoup a partir del HTML

market_soup = soup(html, 'html.parser') """

' # Leer html desde archivo\n\nwith open(nombre_html, "r", encoding="utf-8") as file:\n    html = file.read()\n\n# Crear un objeto BeautifulSoup a partir del HTML\n\nmarket_soup = soup(html, \'html.parser\') '

In [249]:
# Extract all the necessary info and insert into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]
locations_and_mileages_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
locations_and_mileages_list = [location_and_mileage.text.strip() for location_and_mileage in locations_and_mileages_div]
urls_div = market_soup.find_all('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
base_url = "https://web.facebook.com"
urls_list = [base_url+url.get('href') for url in urls_div]

In [250]:
# locations_and_mileages_list

titles_div


[<span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2021 Kia sonet</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2012 Kia morning</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2019 Kia rio</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2019 Kia frontier</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2017 Kia frontier doble cabina barato</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2011 Kia soul</span>,
 <span class="x1lliihq x6ikm8r x10wlt62

In [251]:
# Create a regular expression pattern to match city and state entries like "City, State"
pattern = re.compile(r'(\w+(?:-\w+)?, [A-Z]{2})')

# Define regular expressions to extract numeric mileage values in "K km" and "K miles" format
mileage_pattern_km = r'(\d+) km'
mileage_pattern_miles = r'(\d+) millas'

# Initialize an empty list to store adjusted mileage entries
locations_and_mileages_list2 = []

# Iterate through the original mileage entries
for item in locations_and_mileages_list:
    # Append the current mileage entry to the adjusted list
    locations_and_mileages_list2.append(item)	

    # Check if the current mileage entry matches the pattern and there are at least two entries in the adjusted list
    if item == '':
        locations_and_mileages_list2[-1] = item.replace('', '0')
        print(f"Se encontró un espacio en blanco en el elemento {len(locations_and_mileages_list2)} y se rellenó con 0 km")

    elif pattern.search(item) and len(locations_and_mileages_list2) >= 2 and pattern.search(locations_and_mileages_list2[-2]):
        # If the conditions are met, insert "0K km" in between the two consecutive city and state entries
        locations_and_mileages_list2.insert(-1, '0')
        print(f"Se encontró un espacio en blanco en el elemento {len(locations_and_mileages_list2)} y se rellenó con 0 km")  

    
    # Try to find a match for the "K km" format
    match_mileage_km = re.search(mileage_pattern_km, locations_and_mileages_list2[-1])
        
    # Try to find a match for the "K miles" format
    match_mileage_miles = re.search(mileage_pattern_miles, locations_and_mileages_list2[-1])

    if "\xa0mil" in item: #Por esta parte se necesita que se busquen autos con mas de 1000 km
        locations_and_mileages_list2[-1] = item.replace("\xa0mil", "000")
        # Try to find a match for the "K km" format
        match_mileage_km = re.search(mileage_pattern_km, locations_and_mileages_list2[-1])
        
        # Try to find a match for the "K miles" format
        match_mileage_miles = re.search(mileage_pattern_miles, locations_and_mileages_list2[-1])
         
        
    # Check if either of the formats is found
    if match_mileage_km or match_mileage_miles:
        # If "K km" format is found, convert it to meters and append to the cleaned list
        if match_mileage_km:
            locations_and_mileages_list2[-1] = match_mileage_km.group(1)
        # If "K miles" format is found, convert it to meters and append to the cleaned list
        else:
            locations_and_mileages_list2[-1] = str((int(match_mileage_miles.group(1)) * 1609//1000))
            print(f"Se encontró una distancia en K millas {int(match_mileage_miles.group(1))}, se convirtió a {locations_and_mileages_list2[-1]} metros")

Se encontró una distancia en K millas 149000, se convirtió a 239741 metros
Se encontró una distancia en K millas 154000, se convirtió a 247786 metros
Se encontró una distancia en K millas 128000, se convirtió a 205952 metros
Se encontró un espacio en blanco en el elemento 162 y se rellenó con 0 km
Se encontró una distancia en K millas 148000, se convirtió a 238132 metros
Se encontró una distancia en K millas 50000, se convirtió a 80450 metros
Se encontró una distancia en K millas 148000, se convirtió a 238132 metros
Se encontró una distancia en K millas 8000, se convirtió a 12872 metros
Se encontró una distancia en K millas 150000, se convirtió a 241350 metros
Se encontró una distancia en K millas 123000, se convirtió a 197907 metros
Se encontró una distancia en K millas 104000, se convirtió a 167336 metros
Se encontró una distancia en K millas 65000, se convirtió a 104585 metros
Se encontró una distancia en K millas 120000, se convirtió a 193080 metros
Se encontró una distancia en K m

In [252]:
# Separate locations and mileages lists

# Initialize an empty list to store cleaned values
mileage_clean = []
locations_list = []

# Iterate through the locations_and_mileages_list2 in steps of 2 to separate locations and mileage
for i in range(0, len(locations_and_mileages_list2), 2):
    # Extract location
    location_parts = locations_and_mileages_list2[i].split(',')
    if len(location_parts) > 0:
        # Clean and decode the location to handle UTF-8 characters properly
        location = location_parts[0].strip().encode('utf-8').decode('utf-8')
        locations_list.append(location)   

    # Extract mileage
    mileage_parts = locations_and_mileages_list2[i + 1].split(' ')
    if len(mileage_parts) > 0:
        mileage = mileage_parts[0]
        try:
            mileage_clean.append(int(mileage))
        except:
            print(Fore.RED + locations_and_mileages_list2[i + 1] + Fore.RESET)

print("Locations:", locations_list)
print("Mileage:", mileage_clean)

Locations: ['Padre Hurtado', 'Santiago', 'San Antonio', 'Santiago', 'Santiago', 'Maipú', 'Macul', 'San Bernardo', 'Independencia', 'Melipilla', 'Santiago', 'Santiago', 'Talagante', 'Santiago', 'Peñaflor', 'Maipú', 'Villa Alemana', 'Talagante', 'Maipú', 'Santiago', 'Santiago', 'Santiago', 'Maipú', 'Limache', 'Buin', 'Maipú', 'Viña del Mar', 'Olmué', 'Puente Alto', 'Padre Hurtado', 'Santiago', 'Santiago', 'San Bernardo', 'Viña del Mar', 'San Antonio', 'San Miguel', 'Las Condes', 'Puente Alto', 'Lampa', 'Santiago', 'Villa Alemana', 'Quilicura', 'Quillota', 'Peñalolén', 'San Ramón', 'Coltauco', 'Melipilla', 'Quilicura', 'Villa Alemana', 'La Reina', 'Santiago', 'Puente Alto', 'Chépica', 'Maipú', 'María Pinto', 'Lampa', 'Curacaví', 'San Miguel', 'Cartagena', 'Santiago', 'Las Condes', 'Santiago', 'San Miguel', 'Puente Alto', 'Melipilla', 'San Antonio', 'Santiago', 'Pudahuel', 'Rengo', 'Quilpué', 'Macul', 'Peumo', 'Cartagena', 'Santiago', 'Villa Alemana', 'Villa Alemana', 'Pedro Aguirre Cerda'

In [253]:
prices_list

['$6.700.000',
 '$3.500.000',
 '$5.950.000',
 '$4.000.000',
 '$6.800.000',
 '$3.850.000',
 '$7.700.000',
 '$8.300.000',
 '$6.400.000',
 '$1.900.000',
 '$5.000.000',
 '$5.350.000',
 '$7.500.000',
 '$4.700.000',
 '$2.900.000',
 '$4.170.000',
 '$4.700.000',
 '$3.000.000',
 '$5.600.000',
 '$6.850.000',
 '$5.200.000',
 '$3.850.000',
 '$3.990.000',
 '$4.750.000',
 '$6.700.000',
 '$2.373.000',
 '$6.200.000',
 '$7.500.000',
 '$6.100.000',
 '$6.700.000',
 '$6.300.000',
 '$5.380.000',
 '$5.500.000',
 '$6.700.000',
 '$6.300.000',
 '$1.800.000',
 '$9.900.000',
 '$6.500.000',
 '$2.800.000',
 '$9.250.000',
 '$5.950.000',
 '$3.500.000',
 '$8.000.000',
 '$6.800.000',
 '$3.980.000',
 '$3.000.000',
 '$3.500.000',
 '$4.500.000',
 '$2.500.000',
 '$6.500.000',
 '$5.780.000',
 '$3.850.000',
 '$4.300.000',
 '$6.700.000',
 '$8.800.000',
 '$6.490.000',
 '$8.500.000',
 '$8.500.000',
 '$7.700.000',
 '$5.890.000',
 '$6.380.000',
 '$6.500.000',
 '$5.980.000',
 '$7.100.000',
 '$5.500.000',
 '$8.600.000',
 '$3.000.0

In [254]:
# Ajustar formato según si está en stock o no
if availability == "out%20of%20stock":
    prices_list =  [elemento for elemento in prices_list if elemento != '·']
#prices_list
    
clean_prices_list = []
# Limpiar y convertir los precios
for precio in prices_list:
    # Remover espacios en blanco y símbolo "$", y convertir a entero
    try: 
        precio_limpio = int(precio.replace('\xa0', '').replace('$', ''))
        # Agregar el precio limpio a la lista
        clean_prices_list.append(precio_limpio)
        
    except:
        try: 
            precio_limpio = int(precio.replace('.', '').replace('$', ''))
            # Agregar el precio limpio a la lista
            clean_prices_list.append(precio_limpio)
        except:
            #Delete element from the lists
            index = prices_list.index(precio)
            print(Fore.RED + f"Se encontró un precio no válido: {precio}, en la posición {index}" + Fore.RESET)
            print(urls_list[index])
            del prices_list[index]
            del titles_list[index]
            del locations_list[index]
            del mileage_clean[index]
            del urls_list[index]
        
        

Se encontró un precio no válido: C$3.100.000, en la posición 967
https://web.facebook.com/marketplace/item/356852197328805/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD


In [255]:
clean_prices_list

[6700000,
 3500000,
 5950000,
 4000000,
 6800000,
 3850000,
 7700000,
 8300000,
 6400000,
 1900000,
 5000000,
 5350000,
 7500000,
 4700000,
 2900000,
 4170000,
 4700000,
 3000000,
 5600000,
 6850000,
 5200000,
 3850000,
 3990000,
 4750000,
 6700000,
 2373000,
 6200000,
 7500000,
 6100000,
 6700000,
 6300000,
 5380000,
 5500000,
 6700000,
 6300000,
 1800000,
 9900000,
 6500000,
 2800000,
 9250000,
 5950000,
 3500000,
 8000000,
 6800000,
 3980000,
 3000000,
 3500000,
 4500000,
 2500000,
 6500000,
 5780000,
 3850000,
 4300000,
 6700000,
 8800000,
 6490000,
 8500000,
 8500000,
 7700000,
 5890000,
 6380000,
 6500000,
 5980000,
 7100000,
 5500000,
 8600000,
 3000000,
 5600000,
 600000,
 6100000,
 7400000,
 2800000,
 2850000,
 7800000,
 1100000,
 4600000,
 5100000,
 3700000,
 9550000,
 5500000,
 3900000,
 2950000,
 4700000,
 5000000,
 8800000,
 6000000,
 9100000,
 5890000,
 2500000,
 7900000,
 6990000,
 950000,
 4900000,
 3000000,
 3000000,
 5500000,
 2700000,
 8100000,
 2500000,
 4800000,
 5

In [256]:
def get_description(url):
    driver.get(url)
    
    
    try:
        # Buscar el elemento de la X para ignorar el inicio de sesión
        x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")
        # Hacer clic en la X para ignorar el inicio de sesión
        x_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente
        # time.sleep(random_wait)
    except:
        pass

    try:

        ver_mas_button = driver.find_element(By.XPATH, "//div[contains(@class, 'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x1n2onr6 x87ps6o x1lku1pv x1a2a7pz')]//*[contains(text(), 'Ver más')]")
        # Hacer clic en el botón "Ver más"
        ver_mas_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente
        

    except:
        pass
    
    html = driver.page_source
    item_soup = soup(html, 'html.parser')

    #description_spans = item_soup.find('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u")
    description_spans = item_soup.find('div', class_="xz9dl7a x4uap5 xsag5q8 xkhd6sd x126k92a")

    #Si no se encuentra la descripción, se le asigna "No description"
    if description_spans == None:
        description = "No description"
    else:
        description = description_spans.text.strip()

    words = description.split()  # Dividir la descripción en palabras
    if len(words) >= 2 and words[-2:] == ["Ver", "menos"]:
        # Si las últimas dos palabras son "Ver menos", eliminarlas
        description = " ".join(words[:-2]).strip()

    #print(description)
    time.sleep(random_wait)
    return description


La parte de filtrado de resultados obtenidos es la más demorada de todas

In [257]:
# Filter and add all values to a list of dictionaries
items_list = []

patron = re.compile(r'\b{}\b'.format(re.escape(make)), re.IGNORECASE)

for i, item in enumerate(titles_list):
    items_dict = {}
    
    title_split = titles_list[i].split()

    # Check if the title contains the search make
    if re.search(patron, item):

        items_dict["Titel"] = item
        items_dict["Year"] = int(title_split[0])
        items_dict["Make"] = title_split[1]
        #items_dict["Model"] = title_split[2]
        items_dict["Model"] = ' '.join(title_split[2:])
        items_dict["Price"] = clean_prices_list[i]
        items_dict["Search Model"] = search_model
        items_dict["Transmission"] = transmission
        items_dict["Mileage"] = mileage_clean[i]
        items_dict["Location"] = locations_list[i]
        items_dict["Description"] = get_description(urls_list[i]) #La parte más lenta

        detected_flags = []            

        # Check for flags in the description
        description = unidecode(items_dict["Description"].lower())  # Convertir la descripción a minúsculas para hacer la búsqueda sin distinción de mayúsculas y minúsculas y CREO que para quitar acentos
        for flag in flags:
            if re.search(r'\b{}\b'.format(re.escape(flag)), description):
                if flag == 'detalles' and ('detalles esteticos' in description or 'detalles de pintura' in description or 'paqueños detalles' in description or 'detalles menores' in description or 'detalles minimos' in description):
                    # Si encuentra detalles menores se guarda en los flags como "detalles menores"
                    detected_flags.append('detalles menores')

                if flag == 'multa' and ('ninguna multa' in description or 'sin multa' in description):
                    continue
                elif flag == 'multas' and ('sin multas' in description or 'no tiene multas' in description or 'libre de multas' in description or 'ni multas' in description or 'no multas' in description):
                    continue
                elif flag == 'parte' and ('ningun parte' in description):
                    continue
                elif flag == 'partes' and ('sin partes' in description or 'no tiene partes' in description or 'libre de partes' in description or 'ni partes' in description or 'no partes' in description):
                    continue
                elif flag == 'detalles' and ('sin detalles' in description or 'no tiene detalles'):
                    continue
                elif flag == 'detalle' and ('sin detalle' in description or 'ningun detalle' in description):
                    continue
                elif flag == 'choques' and ('sin choques' in description):
                    continue
                elif flag == 'chocado' and ('nunca chocado' in description):
                    continue
                else:
                    detected_flags.append(flag)

        if detected_flags:
            print(f"[{i}/{len(titles_list)}] Se encontraron los flags {detected_flags} en la siguiente descripción: {items_dict['Description']}")


        items_dict["Flags"] = detected_flags
        items_dict["URL"] = urls_list[i]

        items_list.append(items_dict)
    
#print(items_list)
print("\nSe obtuvieron "+ str(len(items_list)) + " resultados utilizables")
print("De una búsqueda de "+ str(len(titles_list)) +" resultados")


[0/1751] Se encontraron los flags ['al dia', 'multas'] en la siguiente descripción: 🟡VENDO KIA SONET EX 1.5 🟡 💲6.700.000.- AÑO : 2021 KM : 30.000 COLOR : AZUL GRIEGO COMBUSTIBLE : GASOLINA TRANSMISIÓN : MANUAL NO POSEE PRENDA NO POSEE MULTAS NO ACTIVO AIR BAG FUNCIONANDO PAPELES AL DÍA 🟡TRANSFERIBLE ✔️ !!SOLO REALES INTERESADOS !! 📲 📞[hidden information] 🔘SOLO VENTA 🔚🔚🏁
[1/1751] Se encontraron los flags ['al dia'] en la siguiente descripción: Auto Kia morning año 2012 en excelente condiciones como se muestran en las fotografías económico documentacion Al día hasta el año 2025 sin multas solo personas Serias motor en excelentes condiciones a toda Prueba
[2/1751] Se encontraron los flags ['conversable', 'al dia', 'chocado'] en la siguiente descripción: Se vende Kia Río 5 2019 Motor 1.4 mecánico 65.000 km Cierre centralizado Alzavidrios eléctrico Apple CarPlay y Android Auto Detalles: Airbags activos (no operativos) Fue chocado pero reparado Sin A/c Precio vehículo: $5.950.000 Precio merc

In [258]:
# Cerrar la sesión del navegador
#driver.quit()

In [259]:
items_df = pd.DataFrame(items_list)

# Set the display option to ensure that all characters in a column are shown
pd.set_option('display.max_colwidth', None)

items_df.tail()

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,Flags,URL
1252,2013 Kia sportage,2013,Kia,sportage,4990000,,manual,140000,Viña del Mar,Vendo mi camioneta con todos sus papeles y mantenciones al dia Funcionando al 100% O PERMUTO POR vehículo de similares características y valor AUTOMÁTICO,[al dia],https://web.facebook.com/marketplace/item/1138307547199045/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1253,2012 Kia rio,2012,Kia,rio,1790000,,manual,75000,San Joaquín,KIA RIO EX 1.4 2012 👩🏻‍💻Ejecutiva de ventas Stephanie Andrewartha 📲 +569/3924/9889 ➡️ Crédito automotriz (evalúa online) ➡️ Compra al contado o con tarjeta de crédito ➡️ Auto en parte de pago (sujeto a evaluación) Instagram: Ventadeautosst Whatsapp: +569/3924/9889 Valor publicado no incluye la transferencia ➡️ Solicita el catálogo con el stock disponible al whatsapp 📲 +569/3924/9889 ‼️CONSULTA CÓMO RESERVAR‼️( Se venden rápido),[parte],https://web.facebook.com/marketplace/item/7044157375694122/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1254,2010 Kia morning con multas,2010,Kia,morning con multas,5900000,,manual,241000,Rengo,"KIA MORNING 2010 TRANSFERIBLE CON MULTAS 💰1.790.000💰 Vehículo en excelentes condiciones, version semi full, andando a toda prueba, motor y caja impecable, tiene 241.000 KM, estéticamente algunos detalles, rayas superficiales que no afectan en su funcionamiento (nada grave)… VEHÍCULO POSEE MULTAS DE TAG (MUCHAS MULTAS IMPAGABLES) PAPELES DE CIRCULACIÓN AL DIA HASTA 2025…. FAVOR SOLO ENTENDIDOS Y REALES INTERESADOS EN EL VEHÍCULO… NO SE NADA DE SU DEUDA TAG… SE COMPRO CON LA DEUDA… TRANSFERIBLE… tiene las siguientes características: ✅ MOTOR 1.1 ✅ CAJA 5 MECÁNICA ✅ ASIENTOS TELA ISOFIX ✅ BENCINERO ✅ DIRECCIÓN ASISTIDA ✅ ALZAVIDRIOS ELÉCTRICOS ✅ ESPEJOS ABATIBLES ✅ PAPELES AL DÍA 2025 ✅ RADIO FM AUX CD ✅ NEUMATICOS AL 80% ✅ MANTENCIONES AL DÍA ✅ 241.000 KM Se puede ver en rengo… más fotos y detalles por interno…","[al dia, multas]",https://web.facebook.com/marketplace/item/934583371441318/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1255,2013 Kia soul,2013,Kia,soul,6800000,,manual,174000,Constitución,Se vende por renovación * Full * Motor 1.6 * Caja sexta * Papeles al día * Llegar y transferir * Mantenciones al día * Cierre centralizado * Alza vidrio eléctricos * Comando al volante * Vehículo en excelentes condiciones,[al dia],https://web.facebook.com/marketplace/item/393291693456869/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1256,2010 Kia sorento,2010,Kia,sorento,5500000,,manual,289000,Quillota,"Se vende Kia Sorento Diésel año 2010, en excelente estado mecánicamente solo detalles de pintura por el uso, se vende solo por renovación. Se puede ver en Quillota. Precio $6.800.000 conversable","[conversable, detalles menores]",https://web.facebook.com/marketplace/item/1111013403572320/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD


In [260]:
items_df

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,Flags,URL
0,2021 Kia sonet,2021,Kia,sonet,6700000,,manual,30000,Padre Hurtado,🟡VENDO KIA SONET EX 1.5 🟡 💲6.700.000.- AÑO : 2021 KM : 30.000 COLOR : AZUL GRIEGO COMBUSTIBLE : GASOLINA TRANSMISIÓN : MANUAL NO POSEE PRENDA NO POSEE MULTAS NO ACTIVO AIR BAG FUNCIONANDO PAPELES AL DÍA 🟡TRANSFERIBLE ✔️ !!SOLO REALES INTERESADOS !! 📲 📞[hidden information] 🔘SOLO VENTA 🔚🔚🏁,"[al dia, multas]",https://web.facebook.com/marketplace/item/1485533675722687/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1,2012 Kia morning,2012,Kia,morning,3500000,,manual,97000,Santiago,Auto Kia morning año 2012 en excelente condiciones como se muestran en las fotografías económico documentacion Al día hasta el año 2025 sin multas solo personas Serias motor en excelentes condiciones a toda Prueba,[al dia],https://web.facebook.com/marketplace/item/384695987743270/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
2,2019 Kia rio,2019,Kia,rio,5950000,,manual,65000,San Antonio,"Se vende Kia Río 5 2019 Motor 1.4 mecánico 65.000 km Cierre centralizado Alzavidrios eléctrico Apple CarPlay y Android Auto Detalles: Airbags activos (no operativos) Fue chocado pero reparado Sin A/c Precio vehículo: $5.950.000 Precio mercado: $8.900.000 Es muy poco conversable Vehículo está al día en revisión técnica y permiso de circulación 2024 Solo hablo por wsp, si realmente interesad@ pídeme el número","[conversable, al dia, chocado]",https://web.facebook.com/marketplace/item/1444455392871136/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
3,2019 Kia frontier,2019,Kia,frontier,4000000,,manual,150000,Santiago,Se vende en prenda tiene un sonido el motor es el.mas full lo vendo por renovación conbersable,[],https://web.facebook.com/marketplace/item/1484419172495971/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
4,2017 Kia frontier doble cabina barato,2017,Kia,frontier doble cabina barato,6800000,,manual,143000,Santiago,Kia Frontier • Doble Cabina | Año 2017 | 143.822 KM • 2.5 • Diésel • Alza vidrios • Aire Ac • Airbag buenos | 1 Solo Dueño | $6.800.000.- Oferta | Valor Comercial Aprox $12.000.000.- | Transferible | Más info WhatsApp [hidden information],[],https://web.facebook.com/marketplace/item/1807784526391910/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
...,...,...,...,...,...,...,...,...,...,...,...,...
1252,2013 Kia sportage,2013,Kia,sportage,4990000,,manual,140000,Viña del Mar,Vendo mi camioneta con todos sus papeles y mantenciones al dia Funcionando al 100% O PERMUTO POR vehículo de similares características y valor AUTOMÁTICO,[al dia],https://web.facebook.com/marketplace/item/1138307547199045/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1253,2012 Kia rio,2012,Kia,rio,1790000,,manual,75000,San Joaquín,KIA RIO EX 1.4 2012 👩🏻‍💻Ejecutiva de ventas Stephanie Andrewartha 📲 +569/3924/9889 ➡️ Crédito automotriz (evalúa online) ➡️ Compra al contado o con tarjeta de crédito ➡️ Auto en parte de pago (sujeto a evaluación) Instagram: Ventadeautosst Whatsapp: +569/3924/9889 Valor publicado no incluye la transferencia ➡️ Solicita el catálogo con el stock disponible al whatsapp 📲 +569/3924/9889 ‼️CONSULTA CÓMO RESERVAR‼️( Se venden rápido),[parte],https://web.facebook.com/marketplace/item/7044157375694122/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1254,2010 Kia morning con multas,2010,Kia,morning con multas,5900000,,manual,241000,Rengo,"KIA MORNING 2010 TRANSFERIBLE CON MULTAS 💰1.790.000💰 Vehículo en excelentes condiciones, version semi full, andando a toda prueba, motor y caja impecable, tiene 241.000 KM, estéticamente algunos detalles, rayas superficiales que no afectan en su funcionamiento (nada grave)… VEHÍCULO POSEE MULTAS DE TAG (MUCHAS MULTAS IMPAGABLES) PAPELES DE CIRCULACIÓN AL DIA HASTA 2025…. FAVOR SOLO ENTENDIDOS Y REALES INTERESADOS EN EL VEHÍC

In [261]:
csv_name = f"{make.upper()}_{transmission.upper()}.csv"
# Guardar el DataFrame en un archivo CSV
items_df.to_csv(csv_name, index=False)  # El parámetro index=False evita que se agregue una columna de índice al archivo CSV

print(f"Se guardó el DataFrame en el archivo {csv_name}")

Se guardó el DataFrame en el archivo KIA _MANUAL.csv


In [262]:
# Extraer los precios y URLs de la lista de diccionarios
prices = [item["Price"] for item in items_list]
urls = [item["URL"] for item in items_list]

# Calcular el precio promedio
precio_promedio = np.mean(prices)

# Calcular el precio más barato
precio_mas_barato = np.min(prices)
indice_precio_mas_barato = np.argmin(prices)

# Calcular el percentil 10 más bajo
percentil_10_mas_bajo = np.percentile(prices, 10)

# Calcular la mediana de los precios
mediana_precios = np.median(prices)


# Imprimir los resultados
print("Precio promedio:", precio_promedio)
print("Mediana de precios:", mediana_precios)
print("Percentil 10 más bajo:", percentil_10_mas_bajo)
print("Precio más barato:", precio_mas_barato)
print("URL:", urls[indice_precio_mas_barato])

Precio promedio: 5608665.727923628
Mediana de precios: 5800000.0
Percentil 10 más bajo: 2060000.000000001
Precio más barato: 500000
URL: https://web.facebook.com/marketplace/item/3761294890769602/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
